In [ ]:
DATASET_URL = 'https://raw.githubusercontent.com/SASTRA-iGEM2019/ToeholdSwitchDesign/master/data/selected_features.csv'

In [ ]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt  
import math as math
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict 
from scipy.stats import pearsonr
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import metrics
import statsmodels.api as sm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import initializers
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import KFold
from functools import partial


In [ ]:
toehold_data = pd.read_csv(DATASET_URL)
print(np.shape(toehold_data))
target_col = toehold_data.pop('ON/OFF')
toehold_data.insert(9, 'ON/OFF', target_col)
features = toehold_data.iloc[:,:-1]
features_selected = features[['Overall_MFE',
       'Bottom_region_MFE', 'RBS_linker_MFE', 'Net_MFE', 'Freq_MFE_struct',
       'Overall_SH_37']]
target = toehold_data.iloc[:,-1]

In [ ]:
#adjusting r_square calculation for multiple hypothesis testing
#using number of predictors (len(X_train.columns))= 6
def r_square(y_true, y_pred):
  from keras import backend as K
  SS_res =  K.sum(K.square(y_true - y_pred)) 
  SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
  r_squared = (1 - SS_res/(SS_tot + K.epsilon()))
  return (1 - (1-r_squared)*(float(len(y_true)-1))/float(len(y_true)-6-1))



In [ ]:
res=[]
for i in range(100):
  #seed=i
  scaler = StandardScaler()
  X_train, X_test, y_train, y_test = train_test_split(features_selected, target, train_size=0.8, random_state=i, shuffle=True)
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  np.random.seed()
  #print(X_train.shape[1])
  tf.random.set_seed(i)
  HiddenLayer = partial(keras.layers.Dense, activation="sigmoid", kernel_initializer=initializers.RandomUniform(seed=1), kernel_regularizer=keras.regularizers.l1(1e-2))
  #model = keras.models.Sequential()
  #model.add(Dense(6, activation='relu', input_dim=X_train.shape[1]),kernel_regularizer=keras.regularizers.l1(1e-2)) 
  ##model.add(Dense(6, activation='sigmoid')) 
  model = keras.models.Sequential([
                                 keras.layers.InputLayer(6),
                                 HiddenLayer(6),
                                 keras.layers.Dense(1)
  ])
  
  model.compile(loss='mse', optimizer=keras.optimizers.SGD(learning_rate=1e-3), metrics=[r_square])
  earlystopper = EarlyStopping(monitor='val_r_square', patience=5, mode='max', verbose=1, restore_best_weights=True)
  history = model.fit(X_train, y_train, validation_data=(X_test,  y_test), epochs=200, batch_size=32, shuffle=True, callbacks=[earlystopper])
  loss, adj_R_squared = model.evaluate(X_test, y_test, return_dict=False)
  res.append(adj_R_squared)


In [ ]:
#in the argument to load_model() below, replace '/path/to/saved_model/' with the actual path to the saved model folder
#for e.g, as you would find at this link: https://github.com/SASTRA-iGEM2019/ToeholdSwitchDesign/tree/master/NN_Models/Best_Model
miRNA_toehold_feats= pd.read_csv('https://raw.githubusercontent.com/SASTRA-iGEM2019/ToeholdSwitchDesign/master/data/miR_toeholds.csv')
miRNA_toehold_feats = miRNA_toehold_feats.iloc[:,2:]
model = load_model('/path/to/saved_model/',custom_objects={'r_square':r_square})
print(float(model.predict(miRNA_toehold_feats)[0]))


312.143798828125
